<a href="https://colab.research.google.com/github/ConradKash/sunbird_tasks/blob/main/LanguageID_NLP_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Extraction

In [15]:
import pandas as pd
import numpy as np
import json as js
import re
import nltk; nltk.download('wordnet')
import nltk; nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.simplefilter("ignore")

!wget https://raw.githubusercontent.com/SunbirdAI/salt/main/v1.2/salt-test-v1.2.jsonl

data = pd.read_json("salt-test-v1.2.jsonl", lines = True)
data

--2023-07-31 11:19:50--  https://raw.githubusercontent.com/SunbirdAI/salt/main/v1.2/salt-test-v1.2.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217596 (212K) [text/plain]
Saving to: ‘salt-test-v1.2.jsonl.2’

salt-test-v1.2.json 100%[===================>] 212.50K  --.-KB/s    in 0.03s   

2023-07-31 11:19:50 (7.53 MB/s) - ‘salt-test-v1.2.jsonl.2’ saved [217596/217596]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,tts-speech
0,{'eng': 'The fashion industry is starting to t...,NaN
1,{'eng': 'An epidemic is a disease that affects...,NaN
2,{'eng': 'Good quality honey will come from thi...,NaN
3,{'eng': 'She was riding the bicycle while I wa...,NaN
4,{'eng': 'The authorities are looking for two y...,NaN
...,...,...
495,{'eng': 'Developed countries have good medical...,NaN
496,{'eng': 'New Zealand has won the rugby world c...,NaN
497,{'eng': 'Most primary school teachers are not ...,NaN
498,{'eng': 'There is continued heavy gunfire in t...,NaN


# Data Preparation

Dropping non required fields

In [16]:
data = data.drop('tts-speech', axis =1)
print(data)

                                                  text
0    {'eng': 'The fashion industry is starting to t...
1    {'eng': 'An epidemic is a disease that affects...
2    {'eng': 'Good quality honey will come from thi...
3    {'eng': 'She was riding the bicycle while I wa...
4    {'eng': 'The authorities are looking for two y...
..                                                 ...
495  {'eng': 'Developed countries have good medical...
496  {'eng': 'New Zealand has won the rugby world c...
497  {'eng': 'Most primary school teachers are not ...
498  {'eng': 'There is continued heavy gunfire in t...
499  {'eng': 'The little children waved at their pa...

[500 rows x 1 columns]


In [17]:
dataset_dict = {
    "language": [],
    "text": []
}
for row in data['text']:
    for key, value in row.items():
        dataset_dict["language"].append(key)
        dataset_dict["text"].append(value)

In [18]:
df = pd.DataFrame(dataset_dict, columns=['language', 'text'])
df

,language,text
0,eng,The fashion industry is starting to thrive again.
1,lug,Ekisaawe ky'emisono kitandise okusituka nate.
2,ach,Yub me cital ruk mapatpat manyen tye ka dongo ...
3,teo,Ageutu ikampunin luka enape apolo bobo.
4,lgg,Okalamvu suta o'diru 'diyini 'diyi 'ye e'do tutu
...,...,...
2995,lug,Abaana abato baawuubira bazadde baabwe emikono...
2996,ach,Lutino matino giwito cingi me moto lunyodogi k...
2997,teo,Apotu idwe iyogata auriakake nautatar alosit.
2998,lgg,Anzinyiri nde 'diyi ya dri yima tipika yini dr...



# Text Processing
Cleaning the data by removing noise




In [19]:
text = list(dataset_dict['text'])
lemmatizer = WordNetLemmatizer()
corpus = []

for i in range(len(text)):
    r = text[i]
    r = r.lower()
    r = r.split()
    r = [lemmatizer.lemmatize(word) for word in r]
    r = word_tokenize(str(r))
    r = ' '.join(r)
    corpus.append(r)

df['text'] = corpus


df = pd.DataFrame(df)
df.head(20)


,language,text
0,eng,"[ 'the ' , 'fashion ' , 'industry ' , 'is ' , ..."
1,lug,"[ 'ekisaawe ' , `` ky'emisono '' , 'kitandise ..."
2,ach,"[ 'yub ' , 'me ' , 'cital ' , 'ruk ' , 'mapatp..."
3,teo,"[ 'ageutu ' , 'ikampunin ' , 'luka ' , 'enape ..."
4,lgg,"[ 'okalamvu ' , 'suta ' , `` o'diru '' , `` 'd..."
5,nyn,"[ `` eby'eby'afaashoni '' , 'biriyo ' , 'nibig..."
6,eng,"[ 'an ' , 'epidemic ' , 'is ' , ' a ' , 'disea..."
7,lug,"[ 'endwadde ' , 'ebalukawo ' , `` y'endwadde '..."
8,ach,"[ 'can ' , 'mogo ' , 'obedo ' , 'two ' , 'ma '..."
9,teo,"[ 'erai ' , 'adeka ' , 'na ' , 'ikamuni ' , 'i..."


In [20]:

test = list(dataset_dict['language'])
for i in range(len(test)):
    print(test[i])

eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo
lgg
nyn
eng
lug
ach
teo


In [21]:
x = np.array(df["text"])
y = np.array(df["language"])

cv = CountVectorizer(analyzer = 'char',ngram_range=(1,3))
X = cv.fit_transform(x)
print(X.toarray())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=48)

[[25  0  0 ...  0  0  0]
 [17  0  0 ...  0  0  0]
 [34  0  0 ...  0  0  0]
 ...
 [19  0  0 ...  0  0  0]
 [32  0  0 ...  0  0  0]
 [25  0  0 ...  0  0  0]]


In [22]:

logistic_regression = LogisticRegression()

#fit the model using the training data
logistic_regression.fit(X_train,y_train)

#use model to make predictions on test data
y_pred = logistic_regression.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ach       0.99      0.98      0.99       196
         eng       0.99      1.00      1.00       142
         lgg       0.99      1.00      1.00       162
         lug       0.97      0.99      0.98       170
         nyn       0.99      0.97      0.98       164
         teo       1.00      1.00      1.00       156

    accuracy                           0.99       990
   macro avg       0.99      0.99      0.99       990
weighted avg       0.99      0.99      0.99       990



In [23]:
print(pd.crosstab(y_test, y_pred))

col_0  ach  eng  lgg  lug  nyn  teo
row_0                              
ach    193    1    1    1    0    0
eng      0  142    0    0    0    0
lgg      0    0  162    0    0    0
lug      1    0    0  168    1    0
nyn      1    0    0    4  159    0
teo      0    0    0    0    0  156


# Clean the lug-ach, nyn-ach, nyn-lgg, nyn-lug and teo-nyn

In [24]:
model = MultinomialNB()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.997979797979798

In [25]:
user = "Oliwa nawee "
data = cv.transform([user]).toarray()
output = model.predict(data)

print(output)

['lug']
